In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import snowFun
import dask.array
# %matplotlib widget

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "01_rgi60_Alaska", "01_rgi60_Alaska.shp")
rgi_gdf = gpd.read_file(path_rgi, drop='geometry')

In [5]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

### get list of all the glaciers for which we have calculated the snow covered fractions
# select which rgis to analyze
if validation_only:
    folder_sca = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation', 'Band SCFs')
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
else:
    folder_sca = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Band SCFs')
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# load rgi names that have been saved to the classified folder
rgis_to_analyze = list(set( [ i[3:17] for i in os.listdir(folder_sca) if i!='merged.vrt' ] ))
rgis_to_analyze.sort()

In [6]:
skip = 0
for i in range(len(rgis_to_analyze)):
#     if i!=1: continue
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
    
#     # options for skipping
#     if (i+1)<308:continue #option to skip, NEED TO GO BACK TO 241, 277, 280, 290, 291
    
#     if rgiid != "RGI60-01.09162": continue
#     if i not in [30]: continue
#     if rgiid == "RGI60-01.09162": skip=0
#     if skip: continue
#     if (i+1)!=10: continue

    # check if we've already run this glacier. if so, skip
    temp_path = os.path.join(folder_save, 'Annual AAs', 'csv', f"S2_{rgiid}_2018_2022_annual_AAs.csv")
    if os.path.exists(temp_path):
        continue

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area']
    
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga.values[0]} km2")
    
    # open the snow fraction dataframes
    df_snow = pd.read_csv(os.path.join(folder_sca, f"S2_{rgiid}_snow.csv"))#.rename(columns={'Unnamed: 0':"band_n"}).set_index('band_n')
    df_observed = pd.read_csv(os.path.join(folder_sca, f"S2_{rgiid}_observed.csv"))#.rename(columns={'Unnamed: 0':"band_n"}).set_index('band_n')

    # define the columns that we will be manipulating (ie not the z_min, z_max, etc...)
#     meta_columns = ["z_min", "z_max", "total_pixels"]
    meta_columns = ['z_min', 'z_max'] + [ f"total_pixels_{y}" for y in [2018,2019,2020,2021,2022] ]
    data_cols = df_snow.columns.difference(meta_columns)
    
    # calculate fraction of each band that was observed on each date
    # have to do this one year at a time, because the total area in each band changes each year
    df_observed_frac = df_observed[data_cols].copy()
    df_snow_frac = df_snow[data_cols].copy()
    for y in [2018,2019,2020,2021,2022]:
        
        # grab column names from this year
        data_cols_y = [ d for d in data_cols if int(d[:4])==y ]
        
        # grab the area of each band this year
        total_pixels_y = df_snow[ f"total_pixels_{y}" ]
        
        # divide count of snow (and observed) in each band by the total pixels in each band that year
        df_snow_frac[data_cols_y] = df_snow_frac[data_cols_y].divide(df_observed[data_cols_y]).fillna(0)
        df_observed_frac[data_cols_y] = df_observed_frac[data_cols_y].divide(total_pixels_y, axis=0).fillna(0)
    
    # calculate the fraction of the entire glacier surface that is visible on each date
    df_observed_frac_glacier = df_observed[data_cols].sum(axis=0) / df_observed['total_pixels_2018'].sum()
                          
    # calculate the glacier-wide aar on each date (based on observable surface)
    df_aars = df_snow[data_cols].sum(axis=0) / df_observed[data_cols].sum(axis=0)
    # there are instances where a large portion of the ablation zone is nan (mostly debris cover). we may want to do
    # some interpolation or something before calculating aar. eg fill in accumulation above the ela, ablation below
                          


    
#     # calculate the fraction of each band that was observed in each date
#     df_observed_frac = df_observed[data_cols].divide(df_observed["total_pixels"], axis=0)
    
#     # calculate fraction of each band that was snow (divide n_snow by n_observed)
#     df_snow_frac = df_snow[data_cols].divide(df_observed[data_cols])#, axis=0)
    
    
#     # calculate the fraction of the entire glacier surface that is visible on each date
#     df_observed_frac_glacier = df_observed[data_cols].sum(axis=0) / df_observed['total_pixels'].sum()
    
#     # calculate the glacier-wide aar on each date (based on observable surface)
#     df_aars = df_snow[data_cols].sum(axis=0) / df_observed[data_cols].sum(axis=0)
#     # there are instances where a large portion of the ablation zone is nan (mostly debris cover). we should do
#     # some interpolatino or something before calculating aar. eg fill in accumulation above the ela, ablation below
    
    
    
    
    # transition to numpy arrays for a bit. snow fractions >=0.5 become 1 (accumulation zone)
    # everything <0.5 becomes 0 (ablation)
    np_accumulation = df_snow_frac.copy()
    np_accumulation[np_accumulation>=0.5] = 1
    np_accumulation[np_accumulation<0.5] = 0
    
    # Define a kernel that sums the next 5 (4,3,2,1...) values along the 2nd dimension
    kernel5 = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0])
    kernel4 = np.array([1, 1, 1, 1, 0, 0, 0])
    kernel3 = np.array([1, 1, 1, 0, 0])
    kernel2 = np.array([1, 1, 0])
    kernel1 = np.array([1,])
    
    # apply these kernels, find elevation bands that are the start of 5 accumulation bands in a row (or 4,3,2,1)
    all_elas_5 = (np.apply_along_axis(lambda x: np.convolve(x, kernel5, mode='same'), axis=0, arr=np_accumulation))==5
    all_elas_4 = (np.apply_along_axis(lambda x: np.convolve(x, kernel4, mode='same'), axis=0, arr=np_accumulation))==4
    all_elas_3 = (np.apply_along_axis(lambda x: np.convolve(x, kernel3, mode='same'), axis=0, arr=np_accumulation))==3
    all_elas_2 = (np.apply_along_axis(lambda x: np.convolve(x, kernel3, mode='same'), axis=0, arr=np_accumulation))==2
    all_elas_1 = (np.apply_along_axis(lambda x: np.convolve(x, kernel3, mode='same'), axis=0, arr=np_accumulation))==1

    # so the lowest elevation point in each time that is equal to 5 (4,3,2,1) will be the ela
    first_elas_5 = np.argmax(all_elas_5, axis=0).astype(float)
    first_elas_4 = np.argmax(all_elas_4, axis=0).astype(float)
    first_elas_3 = np.argmax(all_elas_3, axis=0).astype(float)
    first_elas_2 = np.argmax(all_elas_3, axis=0).astype(float)
    first_elas_1 = np.argmax(all_elas_3, axis=0).astype(float)
    
    # make sure that if a column has all 0s then we put nan, to allow filling in
    first_elas_5[(all_elas_5.sum(axis=0)==0)] = np.nan
    first_elas_4[(all_elas_4.sum(axis=0)==0)] = np.nan
    first_elas_3[(all_elas_3.sum(axis=0)==0)] = np.nan
    first_elas_2[(all_elas_2.sum(axis=0)==0)] = np.nan
    first_elas_1[(all_elas_1.sum(axis=0)==0)] = np.nan
    
    # get the final elas, by first taking from 5, then 4, then 3
    final_elas = first_elas_5
    final_elas[np.isnan(final_elas)] = first_elas_4[np.isnan(final_elas)]
    final_elas[np.isnan(final_elas)] = first_elas_3[np.isnan(final_elas)]
#     final_elas[np.isnan(final_elas)] = first_elas_2[np.isnan(final_elas)]
#     print(np.count_nonzero(~np.isnan(final_elas)))
#     final_elas[np.isnan(final_elas)] = first_elas_1[np.isnan(final_elas)]
#     print(np.count_nonzero(~np.isnan(final_elas)))
    
    # lastly, if we still have no ela (the entire glacier is ablation) we'll put the highest elevation band as the ela
    # we should find some way to flag this, to make it more clear
    final_elas[np.isnan(final_elas)] = np_accumulation.shape[0]-1
    
    # create series to link ela band # to band elevation
#     bands_zs = (df_snow['z_min']+df_snow['z_max'])/2
    bands_zs = (df_snow['z_min'])
    
#     fig,axs = plt.subplots(1,2, figsize=(10,5))
#     axs[0].imshow(df_snow_frac, vmin=0, vmax=1, cmap="RdBu")
#     axs[1].imshow(df_snow_frac, vmin=0, vmax=1, cmap="RdBu")
#     axs[1].plot(final_elas, c='yellow')
#     axs[0].set_xlabel('Date')
#     axs[0].set_ylabel('Elevation')
#     plt.tight_layout()

    # now that we have our elas for each time step, lets format into a df with date, ela
    ela_df = pd.DataFrame({'date':data_cols})
    ela_df['ela'] = [bands_zs[int(i)] for i in final_elas]
    ela_df['fraction_observed'] = df_observed_frac_glacier.values
    ela_df['aar'] = df_aars.values
    
    # throw out anything that has <XYZ% observed
    ela_df_good = ela_df[ela_df['fraction_observed']>0.8]
    
    # lets also separately get the 'ideal' ela-aar relationship
#     plt.figure()
#     plt.scatter(ela_df['ela'], ela_df['aar'], c=ela_df.index)
    
    # now for each year, get the maximum ela
    annual_best = []
    for y in [2018,2019,2020,2021,2022]:
        
        # decide which of the columns you are going to want to save
        cols_to_save = ['date', 'ela', 'fraction_observed', 'aar']
        
        # subset to this year. note we only consider obs in the july-november time frame
        elas_this_year = ela_df_good[ (ela_df_good['date']>=f'{y}-07-01') & (ela_df_good['date']<f'{y}-12-01')]
        
        # if len=0, we have no good data so we save a null point
        n_obs = len(elas_this_year)
        if n_obs==0:
            annual_best.append(pd.Series({"date":f"{y}-01-01", "ela":-9999, 'fraction_observed':-9999, 'aar':-9999}))
        
        # else we decide which the best of the elas is and save that info
        else:
            
            # find the maximum ela
            ela_max = np.nanmax(elas_this_year['ela'])

            # get all the dates that had this ela
            ela_max_all = elas_this_year[elas_this_year['ela']==ela_max]

            # we'll take the one that has the minimum aar to be the 'best' estimate
            # for example, if the ela is well above the glacier, there are likely many dates with the same ela found
            ela_max_best = ela_max_all.loc[ela_max_all['aar'].idxmax()]
            
            # gotta make sure that this is not an outlier in ELA or AAR
            
            # save this data
            annual_best.append(ela_max_best[cols_to_save])
#     print(annual_best)
            
    ### save all the data here
    
    # function to format metadata and attributes
    def format_xr_to_save(xr_da):
        xr_da.attrs["res"] = (10,10)
        xr_da.attrs["crs"] = "EPSG:3338"
        xr_da.attrs["transform"] = [10,0,0,0,-10,0]
        xr_da.attrs["_FillValue"] = 0
        xr_da.attrs["long_name"] = rgiid
        xr_da.attrs["description"] = "0: nan, 1: ablation, 2: accumulation"
        xr_da.name = "accumulation_area"

        xr_da.x.attrs["units"] = "meters"
        xr_da.y.attrs["units"] = "meters"
        xr_da.x.attrs["long_name"] = 'x'
        xr_da.y.attrs["long_name"] = 'y'

        return xr_da
    
    # set folder paths, etc...
    path_df_all = os.path.join(folder_save, 'Annual AAs', 'csv', f"S2_{rgiid}_2018_2022_annual_AAs.csv")
    path_xr_all = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_2018_2022_annual_AAs.nc")
    path_xr_avg = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA.nc")
    path_tif_avg = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA.tif")
    
    # one year at a time, get the snow distributions
    all_maps = []
    for s in annual_best:
        
        # get date
        d = s['date']
        
        # skip the null years
        if d[5:7]=='01': continue
        
        # open the snow distribution map
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{d[:4]}_daily_AAs_smoothed.nc")
        snow = xr.open_dataset(path_open, chunks={'time':1})['class'].sel(time=d)
        all_maps.append(snow)
    
    # format to save maps as long as there is usable imagery
    if len(all_maps)>0:
        all_maps = xr.concat(all_maps, dim='time')
        average_map = all_maps.median('time', skipna=True).astype('uint8')#+glacier_mask
        save_xr_all = format_xr_to_save(all_maps.astype('uint8'))
        save_xr_avg = format_xr_to_save(average_map)

        # specify compression/encoding
        encoding = {"accumulation_area":{"zlib": True}}#, "spatial_ref":{"zlib": False}}

        # save
        save_xr_all.to_netcdf(path_xr_all, encoding=encoding)
        save_xr_avg.to_netcdf(path_xr_avg, encoding=encoding)
        save_xr_avg.rio.to_raster(raster_path=path_tif_avg, encoding=encoding)

    # always save the csv
    annual_best = pd.DataFrame(annual_best)
    annual_best.to_csv(path_df_all, index=False) # table with annual end-of-summer ela, aar, date, fraction observed

print('All done!')


Starting 173 of 2984: RGI60-01.01394  4.609 km2

Starting 174 of 2984: RGI60-01.01395  3.002 km2

Starting 175 of 2984: RGI60-01.01401  8.792 km2

Starting 176 of 2984: RGI60-01.01402  4.953 km2

Starting 177 of 2984: RGI60-01.01403  8.885 km2

Starting 178 of 2984: RGI60-01.01407  2.696 km2

Starting 179 of 2984: RGI60-01.01411  2.625 km2

Starting 180 of 2984: RGI60-01.01415  18.639 km2

Starting 181 of 2984: RGI60-01.01419  2.983 km2

Starting 182 of 2984: RGI60-01.01420  15.281 km2

Starting 183 of 2984: RGI60-01.01422  5.949 km2

Starting 184 of 2984: RGI60-01.01424  24.482 km2

Starting 185 of 2984: RGI60-01.01425  7.844 km2

Starting 186 of 2984: RGI60-01.01426  2.492 km2

Starting 187 of 2984: RGI60-01.01433  9.26 km2

Starting 188 of 2984: RGI60-01.01436  6.38 km2

Starting 189 of 2984: RGI60-01.01437  10.901 km2

Starting 190 of 2984: RGI60-01.01469  2.705 km2

Starting 191 of 2984: RGI60-01.01470  173.209 km2

Starting 192 of 2984: RGI60-01.01479  5.569 km2

Starting 193 of


Starting 409 of 2984: RGI60-01.03324  5.079 km2

Starting 410 of 2984: RGI60-01.03338  3.548 km2

Starting 411 of 2984: RGI60-01.03339  4.749 km2

Starting 412 of 2984: RGI60-01.03349  26.916 km2

Starting 413 of 2984: RGI60-01.03379  2.578 km2

Starting 414 of 2984: RGI60-01.03380  2.036 km2

Starting 415 of 2984: RGI60-01.03383  4.014 km2

Starting 416 of 2984: RGI60-01.03384  2.656 km2

Starting 417 of 2984: RGI60-01.03385  2.453 km2

Starting 418 of 2984: RGI60-01.03386  2.73 km2

Starting 419 of 2984: RGI60-01.03388  62.126 km2

Starting 420 of 2984: RGI60-01.03390  2.222 km2

Starting 421 of 2984: RGI60-01.03394  2.697 km2

Starting 422 of 2984: RGI60-01.03395  2.811 km2

Starting 423 of 2984: RGI60-01.03396  2.579 km2

Starting 424 of 2984: RGI60-01.03398  3.303 km2

Starting 425 of 2984: RGI60-01.03399  3.12 km2

Starting 426 of 2984: RGI60-01.03400  2.224 km2

Starting 427 of 2984: RGI60-01.03402  2.112 km2

Starting 428 of 2984: RGI60-01.03404  2.15 km2

Starting 429 of 2984


Starting 576 of 2984: RGI60-01.04452  2.534 km2

Starting 577 of 2984: RGI60-01.04453  2.184 km2

Starting 578 of 2984: RGI60-01.04457  6.478 km2

Starting 579 of 2984: RGI60-01.04459  6.03 km2

Starting 580 of 2984: RGI60-01.04463  5.411 km2

Starting 581 of 2984: RGI60-01.04471  2.591 km2

Starting 582 of 2984: RGI60-01.04473  12.282 km2

Starting 583 of 2984: RGI60-01.04481  2.414 km2

Starting 584 of 2984: RGI60-01.04486  4.929 km2

Starting 585 of 2984: RGI60-01.04487  3.34 km2

Starting 586 of 2984: RGI60-01.04506  7.724 km2

Starting 587 of 2984: RGI60-01.04517  2.541 km2

Starting 588 of 2984: RGI60-01.04520  2.946 km2

Starting 589 of 2984: RGI60-01.04525  2.249 km2

Starting 590 of 2984: RGI60-01.04531  16.038 km2

Starting 591 of 2984: RGI60-01.04532  103.945 km2

Starting 592 of 2984: RGI60-01.04534  2.167 km2

Starting 593 of 2984: RGI60-01.04542  25.995 km2

Starting 594 of 2984: RGI60-01.04544  3.355 km2

Starting 595 of 2984: RGI60-01.04554  2.827 km2

Starting 596 of 


Starting 744 of 2984: RGI60-01.05393  38.549 km2

Starting 745 of 2984: RGI60-01.05413  2.603 km2

Starting 746 of 2984: RGI60-01.05419  4.142 km2

Starting 747 of 2984: RGI60-01.05421  3.003 km2

Starting 748 of 2984: RGI60-01.05441  6.844 km2

Starting 749 of 2984: RGI60-01.05448  4.988 km2

Starting 750 of 2984: RGI60-01.05455  2.135 km2

Starting 751 of 2984: RGI60-01.05471  82.317 km2

Starting 752 of 2984: RGI60-01.05472  12.64 km2

Starting 753 of 2984: RGI60-01.05476  2.929 km2

Starting 754 of 2984: RGI60-01.05479  2.118 km2

Starting 755 of 2984: RGI60-01.05487  2.004 km2

Starting 756 of 2984: RGI60-01.05489  4.568 km2

Starting 757 of 2984: RGI60-01.05490  10.56 km2

Starting 758 of 2984: RGI60-01.05496  2.815 km2

Starting 759 of 2984: RGI60-01.05528  3.564 km2

Starting 760 of 2984: RGI60-01.05541  2.261 km2

Starting 761 of 2984: RGI60-01.05543  10.196 km2

Starting 762 of 2984: RGI60-01.05551  2.57 km2

Starting 763 of 2984: RGI60-01.05561  6.544 km2

Starting 764 of 2


Starting 911 of 2984: RGI60-01.07318  2.088 km2

Starting 912 of 2984: RGI60-01.07359  7.472 km2

Starting 913 of 2984: RGI60-01.07360  3.069 km2

Starting 914 of 2984: RGI60-01.07366  6.958 km2

Starting 915 of 2984: RGI60-01.07369  4.713 km2

Starting 916 of 2984: RGI60-01.07370  2.562 km2

Starting 917 of 2984: RGI60-01.07379  2.677 km2

Starting 918 of 2984: RGI60-01.07386  2.03 km2

Starting 919 of 2984: RGI60-01.07390  5.656 km2

Starting 920 of 2984: RGI60-01.07428  2.193 km2

Starting 921 of 2984: RGI60-01.07438  2.046 km2

Starting 922 of 2984: RGI60-01.07445  6.973 km2

Starting 923 of 2984: RGI60-01.07461  5.885 km2

Starting 924 of 2984: RGI60-01.07462  3.7 km2

Starting 925 of 2984: RGI60-01.07463  3.36 km2

Starting 926 of 2984: RGI60-01.07465  3.022 km2

Starting 927 of 2984: RGI60-01.07470  12.392 km2

Starting 928 of 2984: RGI60-01.07485  2.062 km2

Starting 929 of 2984: RGI60-01.07505  4.83 km2

Starting 930 of 2984: RGI60-01.07522  2.5 km2

Starting 931 of 2984: RGI

...


Starting 2252 of 2984: RGI60-01.19487  2.429 km2

Starting 2253 of 2984: RGI60-01.19492  12.531 km2

Starting 2254 of 2984: RGI60-01.19494  6.479 km2

Starting 2255 of 2984: RGI60-01.19495  3.383 km2

Starting 2256 of 2984: RGI60-01.19502  2.043 km2

Starting 2257 of 2984: RGI60-01.19503  4.216 km2

Starting 2258 of 2984: RGI60-01.19509  3.903 km2

Starting 2259 of 2984: RGI60-01.19515  31.701 km2

Starting 2260 of 2984: RGI60-01.19516  6.61 km2

Starting 2261 of 2984: RGI60-01.19522  2.533 km2

Starting 2262 of 2984: RGI60-01.19526  2.761 km2

Starting 2263 of 2984: RGI60-01.19527  2.107 km2

Starting 2264 of 2984: RGI60-01.19528  3.039 km2

Starting 2265 of 2984: RGI60-01.19534  2.794 km2

Starting 2266 of 2984: RGI60-01.19540  3.342 km2

Starting 2267 of 2984: RGI60-01.19542  71.722 km2

Starting 2268 of 2984: RGI60-01.19548  4.796 km2

Starting 2269 of 2984: RGI60-01.19552  22.057 km2

Starting 2270 of 2984: RGI60-01.19557  3.016 km2

Starting 2271 of 2984: RGI60-01.19558  3.915 k


Starting 2416 of 2984: RGI60-01.20312  3.547 km2

Starting 2417 of 2984: RGI60-01.20313  24.221 km2

Starting 2418 of 2984: RGI60-01.20316  3.164 km2

Starting 2419 of 2984: RGI60-01.20319  5.27 km2

Starting 2420 of 2984: RGI60-01.20320  2.295 km2

Starting 2421 of 2984: RGI60-01.20321  26.848 km2

Starting 2422 of 2984: RGI60-01.20323  4.678 km2

Starting 2423 of 2984: RGI60-01.20324  10.118 km2

Starting 2424 of 2984: RGI60-01.20328  7.424 km2

Starting 2425 of 2984: RGI60-01.20334  2.259 km2

Starting 2426 of 2984: RGI60-01.20336  38.775 km2

Starting 2427 of 2984: RGI60-01.20337  7.168 km2

Starting 2428 of 2984: RGI60-01.20341  7.937 km2

Starting 2429 of 2984: RGI60-01.20361  20.523 km2

Starting 2430 of 2984: RGI60-01.20406  4.511 km2

Starting 2431 of 2984: RGI60-01.20417  79.504 km2

Starting 2432 of 2984: RGI60-01.20447  3.382 km2

Starting 2433 of 2984: RGI60-01.20454  2.17 km2

Starting 2434 of 2984: RGI60-01.20455  7.058 km2

Starting 2435 of 2984: RGI60-01.20460  2.243 


Starting 2580 of 2984: RGI60-01.21012  5.377 km2

Starting 2581 of 2984: RGI60-01.21013  65.309 km2

Starting 2582 of 2984: RGI60-01.21015  89.492 km2

Starting 2583 of 2984: RGI60-01.21016  54.523 km2

Starting 2584 of 2984: RGI60-01.21017  28.196 km2

Starting 2585 of 2984: RGI60-01.21051  6.091 km2

Starting 2586 of 2984: RGI60-01.21055  55.698 km2

Starting 2587 of 2984: RGI60-01.21072  2.335 km2

Starting 2588 of 2984: RGI60-01.21080  3.33 km2

Starting 2589 of 2984: RGI60-01.21081  115.277 km2

Starting 2590 of 2984: RGI60-01.21185  2.145 km2

Starting 2591 of 2984: RGI60-01.21230  7.233 km2

Starting 2592 of 2984: RGI60-01.21257  2.394 km2

Starting 2593 of 2984: RGI60-01.21276  2.275 km2

Starting 2594 of 2984: RGI60-01.21305  4.682 km2

Starting 2595 of 2984: RGI60-01.21347  2.024 km2

Starting 2596 of 2984: RGI60-01.21366  6.045 km2

Starting 2597 of 2984: RGI60-01.21367  6.274 km2

Starting 2598 of 2984: RGI60-01.21370  3.264 km2

Starting 2599 of 2984: RGI60-01.21372  41.0


Starting 2748 of 2984: RGI60-01.22981  2.47 km2

Starting 2764 of 2984: RGI60-01.23053  3.063 km2

Starting 2765 of 2984: RGI60-01.23062  2.413 km2

Starting 2766 of 2984: RGI60-01.23070  2.047 km2

Starting 2767 of 2984: RGI60-01.23072  2.808 km2

Starting 2768 of 2984: RGI60-01.23089  2.196 km2

Starting 2769 of 2984: RGI60-01.23090  2.246 km2

Starting 2770 of 2984: RGI60-01.23094  6.501 km2

Starting 2771 of 2984: RGI60-01.23097  2.061 km2

Starting 2772 of 2984: RGI60-01.23099  2.911 km2

Starting 2773 of 2984: RGI60-01.23136  3.937 km2

Starting 2775 of 2984: RGI60-01.23185  6.739 km2

Starting 2776 of 2984: RGI60-01.23187  2.222 km2

Starting 2777 of 2984: RGI60-01.23193  7.675 km2

Starting 2778 of 2984: RGI60-01.23194  2.172 km2

Starting 2779 of 2984: RGI60-01.23195  2.255 km2

Starting 2780 of 2984: RGI60-01.23196  2.483 km2

Starting 2781 of 2984: RGI60-01.23197  2.837 km2

Starting 2782 of 2984: RGI60-01.23199  7.486 km2

Starting 2783 of 2984: RGI60-01.23219  6.5 km2

St


Starting 2966 of 2984: RGI60-01.26742  5.854 km2

Starting 2967 of 2984: RGI60-01.26743  176.235 km2

Starting 2968 of 2984: RGI60-01.26762  4.283 km2

Starting 2969 of 2984: RGI60-01.26794  7.214 km2

Starting 2970 of 2984: RGI60-01.26823  4.907 km2

Starting 2971 of 2984: RGI60-01.26831  3.936 km2

Starting 2972 of 2984: RGI60-01.26833  3.657 km2

Starting 2973 of 2984: RGI60-01.26861  3.545 km2

Starting 2974 of 2984: RGI60-01.26866  2.201 km2

Starting 2975 of 2984: RGI60-01.26940  9.33 km2

Starting 2976 of 2984: RGI60-01.26953  2.866 km2

Starting 2977 of 2984: RGI60-01.26970  3.81 km2

Starting 2978 of 2984: RGI60-01.26986  2.507 km2

Starting 2979 of 2984: RGI60-01.27101  80.219 km2

Starting 2980 of 2984: RGI60-01.27102  101.741 km2

Starting 2981 of 2984: RGI60-01.27103  86.656 km2

Starting 2982 of 2984: RGI60-01.27105  131.574 km2

Starting 2983 of 2984: RGI60-01.27106  2.758 km2

Starting 2984 of 2984: RGI60-01.27110  3.564 km2
All done!


In [ ]:
# lastly, go through all the saved csvs, make a master csv with rgiid, median ela
# all_files = [ i for i in os.listdir(os.path.join(folder_save, 'csv')) if i[-3:]=='csv' ]

# all_rgis = []
# all_elas = []
# all_aars = []

# for f in all_files:
    
#     # open
#     df = pd.read_csv(os.path.join(folder_save, 'csv', f))
    
#     # get rgi
#     all_rgis.append( f[3:17] )
    
#     # get rid of the 'bad' years
#     df = df[df['ela']>0]
    
#     # get median ela, aar
#     all_elas.append( np.nanmedian(df['ela']) )
#     all_aars.append( np.nanmedian(df['aar']) )

# # save everything
# bulk_df = pd.DataFrame({'rgiid':all_rgis, 'ela':all_elas, 'aar':all_aars}) 
# bulk_df.head()
# # bulk_df.to_csv(os.path.join(folder_save, 'Region_04_2018_2022.csv'), index=False)